## Working with Colloquial datasets

Read in the existing 2 data sets and analyze the number of symptoms on each papaer and create a one merged list of symptoms from both data sets.

In [1]:
import pandas as pd
import numpy as np
import xlrd

In [2]:
#Patient Site data
paSi_df = pd.read_excel('/Users/elif/Downloads/OneDrive_1_10-29-2020/patient-site_lableled.xlsx', sheet_name='in', usecols="A,B,C")
#Patience like me site data
plm_df = pd.read_excel('/Users/elif/Downloads/OneDrive_1_10-29-2020/plm_dataset_labeled.xlsx', sheet_name='plm_dataset', usecols="A,B,C")

Use colloq_data_symp_count module to get a dictionary of all the marked symptoms and their counts of how many times they appear in the  tagged dataframe

In [4]:
from collections import Counter
import Colloq_data_symp_count as coll_count
plm_sym_dict = coll_count.symAndCount(plm_df)
paSi_sym_dict = coll_count.symAndCount(paSi_df)

{'relapse': 1,
 'anxiety': 3,
 'fatigued': 1,
 'wanting to sleep': 1,
 'weird pressure around my head': 1,
 'feels like': 1,
 'pressure around my head and temples and even underneath my jaw and above my nose bridge': 1,
 'water in my ear': 1,
 'tension headache': 1,
 'head hurts': 1,
 'constant pressure all around rather than pain': 1,
 'pressure in your head': 1,
 'head rarely hurts': 1,
 'brain inflammation': 1,
 'headache': 6,
 'coughing': 2,
 'anxious': 1,
 'breath a little better': 1,
 'no fever': 1,
 'no chills': 1,
 'slight cough': 2,
 'cough': 11,
 'oxygen levels were good': 1,
 'i was stable': 1,
 'i was released': 1,
 'helped me with my breathing': 1,
 'helps breathing': 1,
 'suffering from shortness of breath': 1,
 'breathing gets better': 1,
 'very difficult time breathing': 1,
 'choking cough': 1,
 'super weak': 1,
 'could not breathe': 1,
 'collapsed': 1,
 'totally exhausts': 1,
 'drop my oxygen sat rates': 1,
 'fatigues': 1,
 'weak': 1,
 'short of breath': 1,
 'got so we

Full dictionary of symptoms collected from both tagged data files that are from Jul, 2020.

In [5]:
new_dict = dict(Counter(plm_sym_dict) + Counter(paSi_sym_dict))
#Sort the dictionary of symptoms along with their value counts
import operator
sorted_d = dict( sorted(new_dict.items(), key=operator.itemgetter(1),reverse=True))
# sorted_d

In [6]:
#Create a data frame for combined symptoms and how many times they appear in both colloquial tagged data
df_sym_freq = pd.DataFrame(list((dict(new_dict)).items()), columns = ['Symptom', 'Frequency'])
#Output the combined symptoms from both colloquial datasets
df_sym_freq.to_csv('sym_freq.csv',index=False)

#List of symptoms from colloquial datasets
colloq_data_syms = list((dict(new_dict)).keys())